In [1]:
import pandas as pd
import numpy as np
import build_reports as br
from scipy.optimize import curve_fit

import plotly.graph_objects as go

import warnings
warnings.filterwarnings('ignore')


In [2]:
#covid_df = pd.read_csv('datasets/covid_df.csv', dtype=str)
covid_df = pd.read_csv('datasets/cases_by_day.csv', dtype=str)
covid_df.rename(columns={'cases':'confirmed'}, inplace=True)
covid_df[['confirmed', 'x', 'y']] = covid_df[['confirmed', 'x', 'y']].astype(int)
#covid_df[['cases', 'deaths', 'recovered']] = covid_df[['confirmed', 'deaths', 'recovered']].astype(int)
covid_df['date'] = pd.to_datetime(covid_df['date'])
covid_df.tail()

,date,confirmed,x,y
121,2020-07-04,1335,121,127092
122,2020-07-05,601,122,127693
123,2020-07-06,703,123,128396
124,2020-07-07,238,124,128634
125,2020-07-08,4,125,128638


In [9]:
path = 'datasets'
category = 'confirmed'

prediction_days = 300
train_days = 90

cases_by_day = covid_df.loc[covid_df[category] > 0]

fig = go.Figure()
fig.add_trace(go.Scatter(x=cases_by_day['date'], y=cases_by_day['y'], line = dict(width=2, dash='dot'), name='Confirmed cases'))
fig.update_layout(
    title_text="Predictions"
)

fig.show()

In [4]:
cases_by_day.dtypes

date         datetime64[ns]
confirmed             int64
x                     int64
y                     int64
dtype: object

In [5]:
#cases_by_day['x'] = np.arange(0, cases_by_day.shape[0])
#cases_by_day['y'] = cases_by_day[category]
#cases_by_day.head()

In [6]:
predictions = br.get_exponential_predictions(cases_by_day, prediction_days, cases_by_day.shape[0])
predictions = br.set_growth_factor(predictions)

inflection_idx, L = br.logaritmic_parameters(predictions)
print((inflection_idx, L))
pred = predictions.iloc[:cases_by_day.shape[0]]

#pred['y'] = pred['y'].apply(lambda n: 1 if n < 0 else n)

fig = go.Figure()
fig.add_trace(go.Scatter(x=pred['x'], line = dict(width=2, dash='dot'), y=pred['y'], name = 'pred'))
fig.update_layout(
    title_text="Predictions"
)

fig.show()

(165, 715097.0866332956)


In [7]:
logistic_df = br.get_logistic_df(cases_by_day)

[2.00427701e+05 1.11661001e+02 5.67231541e-02 3.34852451e+02]


In [8]:
fig = go.Figure()
#fig.add_trace(go.Scatter(x=pred['x'], line = dict(width=2, dash='dot'), y=pred['y'], name = 'pred'))
fig.add_trace(go.Scatter(x=logistic_df['x'], line = dict(width=2, dash='dot'), y=logistic_df['y'], name = 'Fit curve'))
fig.add_trace(go.Scatter(x=cases_by_day['x'], y=cases_by_day['y'], line = dict(width=2, dash='dot'), name='Confirmed cases'))

fig.update_layout(
    title_text="Predictions"
)

fig.show()